In [1]:
from gurobipy import *

In [55]:
# nと初期配置を変更しなければいけない
ship_num=5
stack_num=5
height=5
time = 10

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
T=[i+1 for i in range(time)]

# S,H,P,T,n1,O

In [56]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,2*ship_num,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=100
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

C:\Users\Owner\AppData\Local\Temp\ipykernel_14332\3691883466.py:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [57]:
a=[]
b={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  b[tuple(j)]=O.count(j)

b

{(2, 1, 3, 5, 4): 3,
 (2, 1, 3, 4, 5): 10,
 (2, 1, 4, 3, 5): 5,
 (1, 2, 3, 5, 4): 5,
 (3, 1, 4, 2, 5): 1,
 (1, 2, 3, 4, 5): 30,
 (1, 3, 2, 4, 5): 17,
 (1, 2, 4, 3, 5): 12,
 (1, 3, 4, 2, 5): 3,
 (2, 3, 1, 5, 4): 1,
 (2, 1, 5, 3, 4): 2,
 (3, 2, 1, 4, 5): 4,
 (1, 4, 2, 3, 5): 1,
 (1, 2, 4, 5, 3): 1,
 (2, 3, 1, 4, 5): 3,
 (3, 4, 1, 2, 5): 1,
 (2, 3, 4, 5, 1): 1}

In [58]:
m=Model("IPS6")

alpha=m.addVar(vtype="C")

# 変数の定義
x,z,w,c,d={},{},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for h in H:
    for p in P:
      for t in T:
        w[s,h,p,t]=m.addVar(vtype="C",lb=0)
        z[s,h,p,t]=m.addVar(vtype="C",lb=0)


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      c[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  d[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in range(1,len(T)):
      m.addConstr(quicksum(w[s,h,p,t] for s in S for h in H)==quicksum(z[s,h,p,t] for s in S for h in H))

for t in range(1,len(T)):
   m.addConstr(quicksum(w[s,h,p,t] for s in S for h in H for p in P)<=1)
   m.addConstr(quicksum(z[s,h,p,t] for s in S for h in H for p in P)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(w[s,h,p,t+1] for s in S for h in H for p in P)<=quicksum(w[s,h,p,t] for s in S for h in H for p in P))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)+quicksum(z[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(1,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)+quicksum(w[s,h,p,t] for p in P)+quicksum(z[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+z[s,h,p,t]==x[s,h,p,t+1]+w[s,h,p,t])

for s in S:
  for h in range(1,len(H)):
    for t in range(1,len(T)):
      m.addConstr(quicksum(x[s,h,p,t] for p in P)+quicksum(x[s,h+1,p,t] for p in P)+quicksum(w[s,h,p,t] for p in P)+quicksum(z[s,h+1,p,t] for p in P)+quicksum(z[s,h,p,t] for p in P)<=2)

for s in S:
  for t in range(1,len(T)-1):
    m.addConstr(quicksum(z[s,h,p,t] for h in H for p in P)+quicksum(w[s,h,p,t+1] for h in H for p in P)<=1)

for s in S:
  for p in P:
    for t in range(1,len(T)-1):
      m.addConstr(quicksum(w[s,h,p,t] for h in H)+quicksum(z[s,h,p,t+1] for h in H)<=1)

# for s in S:
#   for h in range(1,len(H)):
#     for p in P:
#       m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

for s in S:
  for h in H:
    m.addConstr(w[s,h,1,len(T)-1]==0)

for s in S:
  for p in range(2,len(P)+1):
    m.addConstr(w[s,1,p,len(T)-1]==0)

for s in S:
  for h in H:
    for p in P:
      m.addConstr(z[s,h,p,len(T)]==0)
      m.addConstr(w[s,h,p,len(T)]==0)

for s in S:
  for h in range(2,len(H)+1):
    for h_ in range(1,h):
      for i,o in enumerate(O):
        for j,p in enumerate(o):
          m.addConstr(c[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h_,k,len(T)] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(d[i]>=quicksum(c[s,h,i] for s in S for h in H if h!=1)-alpha)

In [59]:
# 初期配置の制約

# m.addConstr(x[1,1,8,1]==1)
# m.addConstr(x[1,2,7,1]==1)
# m.addConstr(x[1,3,5,1]==1)
# m.addConstr(x[1,4,3,1]==1)
# m.addConstr(x[1,4,2,1]==1)
# m.addConstr(x[2,1,8,1]==1)
# m.addConstr(x[2,2,6,1]==1)
# m.addConstr(x[2,3,3,1]==1)
# m.addConstr(x[2,4,2,1]==1)
# m.addConstr(x[3,1,8,1]==1)
# m.addConstr(x[3,2,3,1]==1)
# m.addConstr(x[3,3,4,1]==1)
# m.addConstr(x[3,4,1,1]==1)
# m.addConstr(x[4,1,7,1]==1)
# m.addConstr(x[4,2,5,1]==1)
# m.addConstr(x[4,3,4,1]==1)
# m.addConstr(x[4,4,1,1]==1)
# m.addConstr(x[5,1,6,1]==1)
# m.addConstr(x[5,2,4,1]==1)
# m.addConstr(x[6,1,5,1]==1)
# m.addConstr(x[6,2,4,1]==1)
# m.addConstr(x[6,3,2,1]==1)
# m.addConstr(x[6,4,1,1]==1)

# 初期配置の制約

m.addConstr(x[1,1,5,1]==1)
m.addConstr(x[1,2,3,1]==1)
m.addConstr(x[1,3,1,1]==1)
# m.addConstr(x[1,4,3,1]==1)
m.addConstr(x[2,1,5,1]==1)
m.addConstr(x[2,2,4,1]==1)
# m.addConstr(x[2,3,2,1]==1)
# m.addConstr(x[2,4,1,1]==1)
m.addConstr(x[3,1,4,1]==1)
m.addConstr(x[3,2,4,1]==1)
m.addConstr(x[3,3,1,1]==1)
# m.addConstr(x[3,4,O1[2],1]==1)
# m.addConstr(x[3,5,O1[0],1]==1)
m.addConstr(x[4,1,5,1]==1)
m.addConstr(x[4,2,2,1]==1)
m.addConstr(x[4,3,3,1]==1)
# m.addConstr(x[4,4,O1[2],1]==1)
m.addConstr(x[5,1,2,1]==1)
m.addConstr(x[5,2,1,1]==1)
# m.addConstr(x[6,1,O1[7],1]==1)
# m.addConstr(x[6,2,O1[4],1]==1)
# m.addConstr(x[6,3,O1[6],1]==1)
# m.addConstr(x[6,4,O1[3],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [60]:
beta=0.8
# m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))
m.setObjective(alpha+quicksum(d[i] for i in range(len(O)))/((1-beta)*len(O))+10**(-5)*quicksum(z[s,h,p,t] for s in S for h in H for p in P for t in range(1,len(T))))

In [61]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 27269 rows, 5851 columns and 201458 nonzeros
Model fingerprint: 0x6d0e114a
Variable types: 4601 continuous, 1250 integer (1250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 21728 rows and 2613 columns
Presolve time: 0.41s
Presolved: 5541 rows, 3238 columns, 50116 nonzeros
Variable types: 2152 continuous, 1086 integer (1086 binary)

Root relaxation: objective 6.000000e-05, 3082 iterations, 0.33 seconds (0.59 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00006    0  136          -    0.00006      -     -    1s
     0     0    0.00006    0  205          -    0.00006 

In [69]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  # for (s,h,p,t) in x:
  #   if t==len(T):
  #     if x[s,h,p,t].X>EPS:
  #       print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))
  y=[]
  for (s,h,p,t) in z:
    for (s_,h_,p_,t_) in w:
      if z[s,h,p,t].X>EPS:
        if w[s_,h_,p_,t_].X>EPS:
          if t==t_:
            y.append(t)
            y.sort()
            for y_ in y:
              if y_==t:
                print("y[%2s,%2s,%2s,%2s]=%3s" %(s_,s,p,t,z[s,h,p,t].X))
      # print("w[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,w[s,h,p,t].X))

  for (s,h,i) in c:
    if c[s,h,i].X>EPS:
      print("c[%2s,%2s,%2s]=%3s  %4s" %(s,h,i,c[s,h,i].X,O[i]))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

y[ 3, 1, 1, 1]=1.0
y[ 4, 1, 3, 4]=1.0
y[ 5, 2, 1, 3]=1.0
y[ 1, 2, 1, 7]=1.0
y[ 1, 2, 1, 8]=1.0
y[ 2, 3, 4, 2]=1.0
y[ 5, 4, 2, 5]=1.0
y[ 1, 5, 3, 6]=1.0
y[ 1, 5, 3, 9]=1.0
c[ 2, 2,94]=1.0  [2, 3, 4, 5, 1]
c[ 2, 3,94]=1.0  [2, 3, 4, 5, 1]
c[ 2, 4,94]=1.0  [2, 3, 4, 5, 1]
the objective function 0.15008994350257818
VaR= 0.0


In [82]:
result=np.zeros((stack_num,height))
for (s,h,p,t) in x:
    if t==len(T):
        if x[s,h,p,t].X>EPS:
            result[height-h][s-1]=int(p)
result=result.astype(int)
print(result)

[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 1 4 2 0]
 [0 1 4 2 3]
 [5 5 4 5 3]]
